# Windows Partitioning

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Requisitos

Se instalan Spark y Java

In [27]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [28]:
ls -l # check the .tgz is there

total 1173060
drwx------  5 root root      4096 Jan 26 16:28 drive/
drwxr-xr-x  1 root root      4096 Jan 24 18:27 sample_data/
drwxr-xr-x 13 1000 1000      4096 Sep  9 02:08 spark-3.5.0-bin-hadoop3/
-rw-r--r--  1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz
-rw-r--r--  1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz.1
-rw-r--r--  1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz.2


In [29]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [30]:
!pip install -q findspark

Se define el entorno

In [31]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Se inicia Spark


In [32]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Window Partitioning") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [33]:
spark

In [34]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [35]:
# Import sql functions
from pyspark.sql.functions import *

##Descarga de los datasets a usar y correcta visualización

El Notebook busca realizar un análisis, mediante Spark, de diferentes datos de la historia de los mundiales de Fórmula 1 en los últimos 70 años.

In [36]:
# Carga de los datasets utilizados para el ejercicio.
circuitsDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/circuits.csv")
circuitsDF.show(2)
cons_resultsDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/constructor_results.csv")
cons_resultsDF.show(2)
cons_standingsDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/constructor_standings.csv")
cons_standingsDF.show(2)
constructorsDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/constructors.csv")
constructorsDF.show(2)
driver_standingsDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/driver_standings.csv")
driver_standingsDF.show(2)
driversDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/drivers.csv")
driversDF.show(2)
qualyDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/qualifying.csv")
qualyDF.show(2)
racesDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/races.csv")
racesDF.show(2)
statusDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/status.csv")
statusDF.show(2)
resultsDF = spark.read.option("header", "true").csv("/content/drive/MyDrive/Spark/notebooks/Datasets/Formula 1/results.csv")
resultsDF.show(2)

+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
|circuitId| circuitRef|                name|    location|  country|     lat|    lng|alt|                 url|
+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
|        1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|144.968| 10|http://en.wikiped...|
|        2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|101.738| 18|http://en.wikiped...|
+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
only showing top 2 rows

+--------------------+------+-------------+------+------+
|constructorResultsId|raceId|constructorId|points|status|
+--------------------+------+-------------+------+------+
|                   1|    18|            1|    14|    \N|
|                   2|    18|            2|     8|    \N|
+--------------------+---

In [42]:
# Unimos las tablas de drivers y drivers_standings para poder conocer los nombres de los pilotos por ID
drivers_namesDF = driver_standingsDF.join(driversDF, 'driverId', 'inner')
drivers_namesDF.show()

# Hacemos join de tabla results para poder introducir los constructorId.
drivers_teamsIdDF = drivers_namesDF.join(resultsDF,'driverId', 'inner')
drivers_teamsIdDF.show()

# Creamos una tabla que contiene a todos los pilotos, así como todas las escuderías para las que han trabajado.
teams_driversDF = (drivers_teams_numberDF.groupBy('driverRef', 'constructorRef').agg(count("*").alias("numero_fila")))
teams_driversDF.show()

# Filtramos por un corredor único, por ejemplo Fernando Alonso, para conocer en qué escuderías ha competido.
alonsoGodDF = teams_driversDF.filter(col("driverRef") == 'alonso')
alonsoGodDF.show()


+--------+-----------------+------+------+--------+------------+----+----------+------+----+---------+----------+----------+-----------+--------------------+
|driverId|driverStandingsId|raceId|points|position|positionText|wins| driverRef|number|code| forename|   surname|       dob|nationality|                 url|
+--------+-----------------+------+------+--------+------------+----+----------+------+----+---------+----------+----------+-----------+--------------------+
|       1|                1|    18|    10|       1|           1|   1|  hamilton|    44| HAM|    Lewis|  Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|                2|    18|     8|       2|           2|   0|  heidfeld|    \N| HEI|     Nick|  Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|                3|    18|     6|       3|           3|   0|   rosberg|     6| ROS|     Nico|   Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|                4|    18|     5|       4| 

In [46]:
# Conocer el número de carreras comenzadas por los pilotos de Formula 1.
# Hacemos un inner join para poder juntar por ID de piloto el nombre y apellido de los mismos.
drivers_standingsDF = driver_standingsDF.join(driversDF, 'driverId', 'inner')
drivers_standingsDF.show()

# Queremos conocer el número de carreras que han comenzado los pilotos.
total_startedRacesDF = drivers_standingsDF.groupBy('driverId', 'driverRef').agg(count('raceId').alias('total_started_races'))
total_startedRacesDF.show()

# Ordenamos la lista para conocer qué pilotos son los que más carreras han comenzado.
top_started_racesDF = total_startedRacesDF.orderBy(desc('total_started_races'))
top_started_racesDF.show(10)

+--------+-----------------+------+------+--------+------------+----+----------+------+----+---------+----------+----------+-----------+--------------------+
|driverId|driverStandingsId|raceId|points|position|positionText|wins| driverRef|number|code| forename|   surname|       dob|nationality|                 url|
+--------+-----------------+------+------+--------+------------+----+----------+------+----+---------+----------+----------+-----------+--------------------+
|       1|                1|    18|    10|       1|           1|   1|  hamilton|    44| HAM|    Lewis|  Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|                2|    18|     8|       2|           2|   0|  heidfeld|    \N| HEI|     Nick|  Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|                3|    18|     6|       3|           3|   0|   rosberg|     6| ROS|     Nico|   Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|                4|    18|     5|       4| 

In [50]:
#Conocer qué escudería ha obtenido más puntos en la historia de la Formula 1.
# Hacemos un inner join para poder juntar por ID de escudería el nombre de la misma.
constructors_standingDF = constructorsDF.join(cons_standingsDF, 'constructorId', 'inner')
constructors_standingDF.show()

# Queremos conocer el número de puntos totales que tiene cada escudería.
total_pointsDF = constructors_standingDF.groupBy('constructorId', 'name').agg(sum('points').alias('total_scored_points'))
total_pointsDF.show()

# Ordenamos la lista para conocer qué escuderías son las que más puntos han conseguido en la historia de la Formula 1.
top_escuderiasDF = total_pointsDF.orderBy(desc('total_scored_points'))
top_escuderiasDF.show(10)

# Cambiamos el nombre de las variables para una mejor visualización de los datos
top_escuderias_renamedDF = top_escuderiasDF.withColumnRenamed('constructorId', 'ID del Constructor').withColumnRenamed('name', 'Nombre del Constructor').withColumnRenamed('total_scored_points', 'Puntuación total')
top_escuderias_renamedDF.show(10)

+-------------+--------------+-----------+-----------+--------------------+----------------------+------+------+--------+------------+----+
|constructorId|constructorRef|       name|nationality|                 url|constructorStandingsId|raceId|points|position|positionText|wins|
+-------------+--------------+-----------+-----------+--------------------+----------------------+------+------+--------+------------+----+
|            1|       mclaren|    McLaren|    British|http://en.wikiped...|                     1|    18|    14|       1|           1|   1|
|            2|    bmw_sauber| BMW Sauber|     German|http://en.wikiped...|                     2|    18|     8|       3|           3|   0|
|            3|      williams|   Williams|    British|http://en.wikiped...|                     3|    18|     9|       2|           2|   0|
|            4|       renault|    Renault|     French|http://en.wikiped...|                     4|    18|     5|       4|           4|   0|
|            5|    t

In [53]:
# Sacar la vuelta rápida de qualy por circuito, año y qué piloto la tiene.
# Hacemos un inner join para poder juntar por ID de carrera el nombre de las carreras disputadas.
qualy_racesDF = qualyDF.join(racesDF, 'raceId', 'inner')
qualy_racesDF.show()

# Queremos introducir el nombre de los pilotos en el dataframe anteriormente creado.
drivers_qualyDF = qualy_racesDF.join(driversDF, 'driverId', 'inner')
drivers_qualyDF.show()

# Filtrar y extraer de la columna de q3 los valores "\N"
filtered_driver_qualyDF = drivers_qualyDF.filter(drivers_qualyDF['q3'] != '\\N')

# Agrupar por las columnas "q3", "nombre circuito", "driverRef" y "año" de consecución de la vuelta rápida en Clasificación. La columna JOIN no me sirve de nada, pero si no no me dejaba enseñar el DF.
grouped_driver_qualyDF = filtered_driver_qualyDF.groupBy('q3', 'name', 'year', 'driverRef').agg(count('*').alias('count'))
grouped_driver_qualyDF.show()

# Filtrar y sacar por pantalla "qualys" en las que haya ganado Fernando Alonso
alonso_qualy = grouped_driver_qualyDF.filter(col('driverRef') == 'alonso')
alonso_qualy.show(3)

+------+---------+--------+-------------+------+--------+--------+--------+--------+----+-----+---------+--------------------+----------+--------+--------------------+--------+--------+--------+--------+--------+--------+----------+----------+-----------+-----------+
|raceId|qualifyId|driverId|constructorId|number|position|      q1|      q2|      q3|year|round|circuitId|                name|      date|    time|                 url|fp1_date|fp1_time|fp2_date|fp2_time|fp3_date|fp3_time|quali_date|quali_time|sprint_date|sprint_time|
+------+---------+--------+-------------+------+--------+--------+--------+--------+----+-----+---------+--------------------+----------+--------+--------------------+--------+--------+--------+--------+--------+--------+----------+----------+-----------+-----------+
|    18|        1|       1|            1|    22|       1|1:26.572|1:25.187|1:26.714|2008|    1|        1|Australian Grand ...|2008-03-16|04:30:00|http://en.wikiped...|      \N|      \N|      \N|  